In [2]:
import pandas as pd


In [ ]:
drop_columns = [ #list of columns to be dropped as they are unnecesary
    "Flow ID",    
    'Fwd Header Length.1',
    "Source IP", "Src IP",
    "Source Port", "Src Port",
    "Destination IP", "Dst IP",
    "Destination Port", "Dst Port",
    "Timestamp",
    "Unnamed: 0", "Inbound", "SimillarHTTP" # CIC-DDoS other undocumented columns
]
df.drop(columns=drop_columns, inplace=True, errors='ignore') 

In [8]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

# Load the dataset
df = pd.read_csv("01-12\\Syn.csv")



C:\Users\Arnav Dham\AppData\Local\Temp\ipykernel_16420\1708296956.py:5: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("01-12\\Syn.csv")


In [11]:
print(df.columns)


Index(['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Le

In [12]:
df.drop(columns=drop_columns, inplace=True, errors='ignore')    
df['Label'].replace({'BENIGN': 'Benign'}, inplace=True)

C:\Users\Arnav Dham\AppData\Local\Temp\ipykernel_16420\3508891651.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Label'].replace({'BENIGN': 'Benign'}, inplace=True)


In [10]:
df.columns = df.columns.str.strip()  # Removes leading/trailing spaces

In [13]:
# Identify benign and attack samples
benign_df = df[df['Label'] == 'Benign']
attack_df = df[df['Label'] != 'Benign']

# Number of benign samples
benign_count = len(benign_df)

# To get a 9:1 ratio, we need attack samples = 9 * benign_count
attack_count_needed = 9 * benign_count

# Downsample the attack class to match the required count
attack_df = attack_df.sample(n=attack_count_needed, random_state=42)

# Combine the balanced attack and benign data
balanced_df = pd.concat([benign_df, attack_df])

# Shuffle the dataset
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the balanced dataset
balanced_df.to_csv("01-12-balanced_dataset.csv", index=False)

print(f"Class balance after resampling: {balanced_df['Label'].value_counts()}")

Class balance after resampling: Label
Syn       3528
Benign     392
Name: count, dtype: int64


In [19]:
import pandas as pd

# Load the balanced dataset from V1
df = pd.read_csv("01-12-balanced_dataset.csv")

# Remove duplicate records
df = df.drop_duplicates()

# Drop columns where all values are missing
df = df.dropna(axis=1, how='all')

# Convert all numeric columns properly
for col in df.columns:
    if df[col].dtype == 'object':  # If column is a string
        try:
            df[col] = pd.to_numeric(df[col])  # Convert to numeric if possible
        except ValueError:
            pass  # Ignore errors and keep as string

# Drop rows with missing values
df = df.dropna()

# Handle infinite values by replacing them with NaN
df.replace([float('inf'), float('-inf')], float('nan'), inplace=True)

# Optionally, drop rows with NaN values after replacing infinities
df = df.dropna()

# Save as CSV
df.to_csv("01-12-cleaned_dataset.csv", index=False)

print("Data cleaning completed. File saved as 01-12-cleaned_dataset.csv")


Data cleaning completed. File saved as 01-12-cleaned_dataset.csv


In [20]:
import pandas as pd

# Load the dataset
df = pd.read_csv("01-12-balanced_dataset.csv")

# Check unique values in the Protocol column
print("Unique values in 'Protocol' before cleaning:", df['Protocol'].unique())

# Remove duplicate records
df = df.drop_duplicates()

# Drop columns where all values are missing
df = df.dropna(axis=1, how='all')

# Convert all numeric columns properly
for col in df.columns:
    if df[col].dtype == 'object':  # If column is a string
        try:
            df[col] = pd.to_numeric(df[col])  # Convert to numeric if possible
        except ValueError:
            pass  # Ignore errors and keep as string

# Check unique values in the Protocol column after cleaning
print("Unique values in 'Protocol' after cleaning:", df['Protocol'].unique())

# Drop rows with missing values
df = df.dropna()

# Handle infinite values by replacing them with NaN
df.replace([float('inf'), float('-inf')], float('nan'), inplace=True)

# Optionally, drop rows with NaN values after replacing infinities
df = df.dropna()

# Save as CSV
df.to_csv("01-12-cleaned_dataset.csv", index=False)

print("Data cleaning completed. File saved as 01-12-cleaned_dataset.csv")


Unique values in 'Protocol' before cleaning: [ 6 17  0]
Unique values in 'Protocol' after cleaning: [ 6 17  0]
Data cleaning completed. File saved as 01-12-cleaned_dataset.csv


In [21]:
print(df[['Protocol']].head(10))  # Print the first 10 rows to inspect the Protocol values


    Protocol
0          6
1          6
6          6
10         6
11         6
12         6
13         6
17         6
18        17
19         6
